In [0]:
import pandas as pd
uriage_data = pd.read_csv('uriage.csv')
uriage_data.head()

In [0]:
kokyaku_data = pd.read_excel('./kokyaku_daicho.xlsx')
kokyaku_data.head()

In [0]:
uriage_data['item_name'].head()

In [0]:
uriage_data['item_price'].head()

In [0]:
kokyaku_data['登録日'].head()

In [0]:
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime('%Y%m')
res = uriage_data.pivot_table(index='purchase_month',columns='item_name',aggfunc='size')
res

In [0]:
res = uriage_data.pivot_table(index='purchase_month', columns='item_name',values='item_price',aggfunc='sum',fill_value=0)
res

In [0]:
len(pd.unique(uriage_data['item_name']))

99

In [0]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
uriage_data['item_name'] = uriage_data['item_name'].str.replace('　','')
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ','')
uriage_data.sort_values(by=['item_name'], ascending=True)

In [0]:
print(pd.unique(uriage_data['item_name']))
print(len(pd.unique(uriage_data['item_name'])))

In [0]:
uriage_data.isnull().any(axis=0)

In [0]:
list(uriage_data.loc[flg_is_null, 'item_name'].unique())

In [0]:
flg_is_null = uriage_data['item_price'].isnull()
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
  price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price' ].max()
  uriage_data['item_price'].loc[(flg_is_null) & (uriage_data['item_name'] == trg)] = price
uriage_data.head()

In [0]:
uriage_data.isnull().any(axis=0)

In [0]:
for trg in list(uriage_data['item_name'].sort_values().unique()):
  print(trg + 'の最大額：' + str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].max()) + 'の最小額：' + str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].min(skipna=False)))

In [0]:
kokyaku_data['顧客名'].head()

In [0]:
uriage_data['customer_name'].head()

In [0]:
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace('　','')
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace(' ','')
kokyaku_data['顧客名'].head()

In [0]:
flg_is_serial = kokyaku_data['登録日'].astype('str').str.isdigit()
flg_is_serial.sum()

22

In [0]:
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial,'登録日'].astype('float'),unit='D') + pd.to_datetime('1900/01/01')
fromSerial

In [0]:
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial,'登録日'])
fromString

In [0]:
kokyaku_data['登録日'] = pd.concat([fromSerial,fromString])
kokyaku_data

In [0]:
kokyaku_data['登録年月'] = kokyaku_data['登録日'].dt.strftime('%Y%m')
rslt = kokyaku_data.groupby('登録年月').count()['顧客名']
print(rslt)
print(len(kokyaku_data))

In [0]:
flg_is_serial = kokyaku_data['登録日'].astype('str').str.isdigit()
flg_is_serial.sum()

0

In [0]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name',right_on='顧客名', how='left')
join_data = join_data.drop('customer_name', axis=1)
join_data

In [0]:
dump_data = join_data[['purchase_date', 'purchase_month', 'item_name', 'item_price','顧客名','かな','地域','メールアドレス','登録日']]
dump_data

In [0]:
dump_data.to_csv('dump_data1.csv', index=False)

In [0]:
import_data = pd.read_csv('dump_data1.csv')
import_data

In [0]:
byItem = import_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
byItem

In [0]:
byPrice = import_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
byPrice

In [0]:
byCustomer = import_data.pivot_table(index='purchase_month', columns='顧客名', aggfunc='size', fill_value=0)
byCustomer

In [0]:
byRegion = import_data.pivot_table(index='purchase_month', columns='地域', aggfunc='size', fill_value=0)
byRegion

In [0]:
away_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='顧客名', how='right')
away_data[away_data['purchase_date'].isnull()][['顧客名','メールアドレス','登録日']]